Install the following libraries:

In [1]:
# TODO: Move dependencies to a Poetry file.

# !pip install -q -U bitsandbytes
# !pip install -q -U git+https://github.com/huggingface/transformers.git
# !pip install -q -U git+https://github.com/huggingface/peft.git
# !pip install -q -U git+https://github.com/huggingface/accelerate.git
# !pip install -q -U datasets
# !pip install -q -U trl
# !pip install -q -U einops

In [2]:
# !pip install -q -U scipy

In [3]:
%load_ext autoreload
%autoreload 2

Import only what we need:

In [5]:
import sys
sys.path.append("../modules")

from datasets import load_dataset
from transformers import TrainingArguments
from trl import SFTTrainer

from training import models

## Fine-Tune Model

Create and prepare the model and dataset:

In [3]:
model, tokenizer, peft_config = models.build_qlora_model()
# TODO: Do I need this?
model.config.use_cache = (
    False  # Gradient checkpointing is used by default but not compatible with caching
)
dataset = load_dataset("timdettmers/openassistant-guanaco", split="train")

Loading checkpoint shards: 100%|██████████| 2/2 [00:10<00:00,  5.38s/it]
Found cached dataset json (/home/pauliusztin/.cache/huggingface/datasets/timdettmers___json/timdettmers--openassistant-guanaco-6126c710748182cf/0.0.0/8bb11242116d547c741b2e8a1f18598ffdd40a1d4f2a2872c7a28b697434bc96)


In [11]:
dataset

Dataset({
    features: ['text'],
    num_rows: 9846
})

In [13]:
dataset["text"][0]

'### Human: Can you write a short introduction about the relevance of the term "monopsony" in economics? Please use examples related to potential monopsonies in the labour market and cite relevant research.### Assistant: "Monopsony" refers to a market structure where there is only one buyer for a particular good or service. In economics, this term is particularly relevant in the labor market, where a monopsony employer has significant power over the wages and working conditions of their employees. The presence of a monopsony can result in lower wages and reduced employment opportunities for workers, as the employer has little incentive to increase wages or provide better working conditions.\n\nRecent research has identified potential monopsonies in industries such as retail and fast food, where a few large companies control a significant portion of the market (Bivens & Mishel, 2013). In these industries, workers often face low wages, limited benefits, and reduced bargaining power, lead

In [6]:
training_arguments = TrainingArguments(
    output_dir="./results",
    per_device_train_batch_size=12,  # increase this value if you have more VRAM
    gradient_accumulation_steps=12,
    optim="paged_adamw_32bit",  # This parameter activate QLoRa's pagination
    save_steps=10,
    logging_steps=10,
    learning_rate=2e-4,
    fp16=True,
    max_grad_norm=0.3,
    num_train_epochs=1,
    warmup_ratio=0.03,
    lr_scheduler_type="constant",
)

Train the model on the guanaco dataset:
*It should complete within 5 hours.*

In [7]:
# trainer = SFTTrainer(
#     model=model,
#     train_dataset=dataset,
#     peft_config=peft_config,
#     dataset_text_field="text",
#     max_seq_length=512,
#     tokenizer=tokenizer,
#     args=training_arguments,
#     packing=True,
# )
# trainer.train()

## Test Fine-Tuned Model

In [6]:
loaded_model, loaded_tokenizer, _ = models.build_qlora_model(peft_model_id="results/checkpoint-10/")

Loading checkpoint shards: 100%|██████████| 2/2 [00:09<00:00,  4.58s/it]


In [10]:
print(
    models.prompt(
        model=loaded_model,
        tokenizer=loaded_tokenizer,
        input_text="What is the meaning of life?",
    )
)

Setting `pad_token_id` to `eos_token_id`:11 for open-end generation.


What is the meaning of life?
The meaning of life is to be happy, to be content, to be fulfilled.
What is the meaning of life?
The meaning of life is to be happy, to be content,
